<h1><b>Machine Learning Based Clustering and Segmentation for Navigation<b></h1>

<h3><b>Introduction</b></h3>
    <p>
    An ML based navigation algorithm that is based on several factors pertaining to neighbourhoods. That will give you the most efficient route to the desired destination, based on factors such as crime rate and population density.
    </p>
<h3><b>Project Contribution</b></h3>
    <p>
    The project contribution is to find correlations between topics surrounding the crime rate, population information and income sources. The purpose of this Jupyter notebook is to focus on the following correlations:
        <ul>
            <li>Correlation between police stations and assault rates</li>
            <li>Correlation between schools and income</li>
        </ul>
    </p>
<h3><b>Prerequisites</b></h3>
<ul>
    <li>Foursquare API</li>
</ul>
<h3><b>Datasets Used</b></h3>

<h3><b>Import Statements</b></h3>

In [3996]:
from dotenv import load_dotenv
from dotenv import dotenv_values
import folium
import requests
import pandas as pd 
from pandas import json_normalize
from bs4 import BeautifulSoup as bs
import os
from sklearn.cluster import KMeans

import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


<h3><b>Download Datasets</b></h3>


In [3997]:
path = os.getcwd()
path = os.path.join(path,"datasets/neighborhood-data.csv")
postcodes = pd.read_csv(path)
postcodes.drop(postcodes.columns[[0]], axis=1, inplace=True)
postcodes.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


<h3><b>Gather the Dataframe for Homicide Rates and Crime Rates</b></h3>

In [3998]:
path2 = os.getcwd()
path2 = os.path.join(path2,"datasets/neighbourhood-crime-rates.csv")
crimedata = pd.read_csv(path2)
crimedata.drop(crimedata.columns[[0]], axis=1, inplace=True)

path3 = os.getcwd()
path3 = os.path.join(path3,"datasets/combined-dataset.csv")
combined_data = pd.read_csv(path3)
combined_data.drop(combined_data.columns[[0]], axis=1, inplace=True)
combined_data.head()

,Postcode,Borough,Neighbourhood,Hood_ID,Population,Assault_AVG,Assault_Rate_2019,AutoTheft_AVG,AutoTheft_Rate_2019,Homicide_AVG,Homicide_Rate_2019,Latitude,Longitude
0,M3A,North York,Parkwoods,45,34805,159.7,454.0,31.5,91.9,0.3,2.9,43.751,-79.323
1,M4A,North York,Victoria Village,43,17510,119.3,753.9,16.5,102.8,0.7,5.7,43.735,-79.312
2,M6A,North York,Lawrence Heights,32,22372,104.0,518.5,28.5,102.8,0.2,0.0,43.722,-79.451
3,M1B,Scarborough,Rouge,131,46496,173.3,391.4,50.5,187.1,0.8,0.0,43.804,-79.165
4,M1B,Scarborough,Malvern,132,43794,278.2,760.4,47.2,162.1,1.7,2.3,43.809,-79.221


<h3><b>Gather the Dataframe for Income Rates and Education Rates</b></h3>

In [3999]:
path4 = os.getcwd()
path4 = os.path.join(path4,"datasets/population-dataset-combined.csv")
population_data = pd.read_csv(path4)
population_data.drop(population_data.columns[[0]], axis=1, inplace=True)
population_data.head()

,Postcode,Borough,Neighbourhood,Neighbourhood Number,"Total - Highest certificate, diploma or degree for the population aged 15 years and over in private households - 25% sample data","No certificate, diploma or degree",Secondary (high) school diploma or equivalency certificate,Trades certificate or diploma other than Certificate of Apprenticeship or Certificate of Qualification,Certificate of Apprenticeship or Certificate of Qualification,"College, CEGEP or other non-university certificate or diploma",...,"$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over","$200,000 and over",Latitude,Longitude
0,M3A,North York,Parkwoods,45,28890,4140,7660,700,605,5295,...,620,"1,200","1,025",880,790,650,3795,890,43.751,-79.323
1,M6A,North York,Lawrence Manor,32,17080,2675,4340,505,330,2635,...,335,735,565,500,435,315,2155,755,43.726,-79.436
2,M1B,Scarborough,Rouge,131,38125,6580,11740,1020,635,7740,...,455,970,950,930,940,845,6060,1075,43.805,-79.166
3,M1B,Scarborough,Malvern,132,35885,7345,11575,1155,710,6915,...,655,"1,360","1,200","1,000",905,795,3280,225,43.809,-79.222
4,M3B,North York,Don Mills North,42,23390,2295,5150,450,345,3490,...,400,930,885,780,655,605,4615,1750,43.761,-79.411


<h3><b>Combine Datafames Together</b></h3>

In [4000]:
k = 6
combined_dataframe = pd.merge(combined_data, population_data, on="Neighbourhood")
combined_dataframe
toronto_clustering = combined_dataframe[["Homicide_AVG", "No certificate, diploma or degree", "Hood_ID", "Neighbourhood Number"]]
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
toronto_clustering
combined_dataframe.insert(0, "Cluster Labels", kmeans.labels_)
combined_dataframe.tail()

,Cluster Labels,Postcode_x,Borough_x,Neighbourhood,Hood_ID,Population,Assault_AVG,Assault_Rate_2019,AutoTheft_AVG,AutoTheft_Rate_2019,...,"$45,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over","$200,000 and over",Latitude_y,Longitude_y
53,0,M8W,Etobicoke,Alderwood,20,12054,36.3,298.7,16.2,116.1,...,165,335,300,320,275,250,1915,360,43.602,-79.545
54,3,M8W,Etobicoke,Long Branch,19,10084,61.7,585.1,12.3,119.0,...,230,400,360,310,295,230,1265,300,43.593,-79.538
55,3,M4X,Downtown Toronto,Cabbagetown,71,11669,102.3,1079.8,10.7,188.5,...,255,425,420,350,295,270,1925,700,43.665,-79.369
56,0,M4X,Downtown Toronto,St. James Town,71,11669,102.3,1079.8,10.7,188.5,...,490,815,600,525,400,340,1170,170,43.671,-79.373
57,4,M8Y,Etobicoke,Mimico NE,17,33964,299.2,959.8,37.3,176.7,...,675,"1,270","1,255","1,120","1,105",920,5455,1245,43.614,-79.495


<h3><b>Foursquare API Initialization / Check</b></h3>
<h4><b>Category Codes:</b></h4>
<ul>
    <li>10000 - Arts and Entertainment</li>
    <li>11000 - Business and Professional Services</li>
    <li>12000 - Community and Government</li>
    <li>13000 - Dining and Drinking</li>
    <li>14000 - Event</li>
    <li>15000 - Health and Medicine</li>
    <li>16000 - Landmarks and Outdoors</li>
    <li>17000 - Retail</li>
    <li>18000 - Sports and Recreation</li>
    <li>19000 - Travel and Transportation</li>
</ul>

In [4052]:
config = dotenv_values(".env")
url = "https://api.foursquare.com/v3/places/nearby"

headers = {"Accept": "application/json",
            "Authorization": config["API_KEY"]}

response = requests.request("GET", url, headers=headers)

def create_request(coords= None, location = None, categories = None, limit = "10"):
    """
        Important:
            - Coords and location cannot be entered together
            - Location and radius cannot be entered together

        The coords will be a list with latitude and longitude.\n 
        Location will be a city and province such as  "Oshawa, ON".\n
        The category is a string from the above codes, with a default of None.\n
        The limit parameter is a maximum of 50, with a default of 10 requests.\n

        Examples:
            - create_request(coords=[-72.848752,43.895962], limit="1")
            - create_request(coords=[-72.848752,43.895962], categories="10000", limit="2")\n
            - create_request(location=["Oshawa","ON"], limit="2")
            - create_request(location=["Oshawa","ON"], categories="10000", limit="20")
    """

    if(coords and categories == None):
        url = "https://api.foursquare.com/v3/places/search?ll=" + str(coords[0]) + "%2C" + str(coords[1]) + "&radius=500"  + "&limit=" + limit
    elif(coords and categories):
        url = "https://api.foursquare.com/v3/places/search?ll=" + str(coords[0]) + "%2C" + str(coords[1]) +"&categories=" + categories + "&radius=500" + "&limit=" + limit
    elif(location and categories == None):
        url = "https://api.foursquare.com/v3/places/search?" + "near=" + str(location[0]) + "%2C" + str(location[1]) + "&limit=" + limit
    elif(location and categories):
        url = "https://api.foursquare.com/v3/places/search?" + "categories=" + categories + "&near=" + str(location[0]) + "%2C" + str(location[1]) + "&limit=" + limit
    else:
        return False
    
    response = requests.request("GET", url, headers=headers)
    
    if(response.status_code == 200):
        return response.json()
    else:
        return False

In [4002]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h3><b>Creating Law Enforcement DataFrame</b></h3>

In [4053]:
latitude = 43.6532
longitude = -79.3832
results = create_request(location=["Toronto","ON"], categories="12070", limit="10")

venues = json_normalize(results['results'], max_level=3)

filtered_columns = ['name', 'categories', 'geocodes.main.latitude', 'geocodes.main.longitude']
venues['categories'] = venues.apply(get_category_type, axis=1)

venues = venues[filtered_columns]
venues.head()

,name,categories,geocodes.main.latitude,geocodes.main.longitude
0,53 Division Toronto Police Service,Police Station,43.706104,-79.400647
1,Toronto Police Service - 13 Division,Police Station,43.698433,-79.436581
2,Toronto Fire Station 341,Fire Station,43.694398,-79.441081
3,Toronto Fire Station 131,Fire Station,43.726175,-79.402729
4,University of Toronto Campus Community Police,Police Station,43.664817,-79.400841


<h3><b>This fucking sucks</b></h3>

In [4059]:
def get_venues(names, latitudes, longitudes):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        results = create_request(location=[lat,lng], limit="50")
        venues = results['results']

        if(len(venues) > 0):
            try:
                venues_list.append([(
                name, 
                lat, 
                lng, 
                venue['name'], 
                venue['geocodes']['main']['latitude'], 
                venue['geocodes']['main']['longitude'], venue['categories'][0]['name']) for venue in venues])
            except:
                continue
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
   
    nearby_venues.columns = ['Neighbourhood','Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude',  'Venue Category']
    return(nearby_venues)


<h3><b>This fucking sucks Part 2</b></h3>

In [4060]:
toronto_venues = get_venues(names=combined_dataframe["Neighbourhood"], latitudes= combined_dataframe["Latitude_y"], longitudes=combined_dataframe["Longitude_y"])
toronto_venues.head()

Parkwoods
Rouge
Malvern
Don Mills North
Humewood-Cedarvale
Eringate
Markland Wood
Guildwood
Morningside
West Hill
The Beaches
Caledonia-Fairbanks
Woburn
Leaside
Hillcrest Village
Bathurst Manor
Thorncliffe Park
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
York University
Little Portugal
Trinity
Ionview
Bayview Village
Riverdale
Oakridge
Bedford Park
Keelesdale
Mount Dennis
Humberlea
Lawrence Park
The Junction North
Dorset Park
Wexford
The Annex
Roncesvalles
Kingsview Village
Martin Grove Gardens
Agincourt
University of Toronto
Runnymede
Sullivan
Tam O'Shanter
Chinatown
Milliken
South Niagara
New Toronto
Beaumond Heights
Jamestown
L'Amoreaux West
Alderwood
Long Branch
Cabbagetown
St. James Town
Mimico NE


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.751,-79.323,Bruno's valu-mart,43.746133,-79.324828,Grocery Store / Supermarket
1,Parkwoods,43.751,-79.323,Brookbanks Park,43.754751,-79.328439,Park
2,Parkwoods,43.751,-79.323,Allwyn's Bakery,43.745806,-79.324683,Bakery
3,Parkwoods,43.751,-79.323,Tim Hortons,43.752874,-79.313900,Restaurant
4,Parkwoods,43.751,-79.323,TD Bank Financial Group,43.745349,-79.324798,Bank


In [4061]:
toronto_venues.groupby("Neighbourhood").count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 257 uniques categories.


<h3><b>Analyzing Each Neigborhood</b></h3>

In [4064]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Neighbourhood,ATM,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Amusement Park,Antique Store,Arcade,Art Gallery,...,Urban Park,Vegan and Vegetarian Restaurant,Video Games Store,Vietnamese Restaurant,Vintage and Thrift Store,Warehouse / Wholesale Store,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4065]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,ATM,Advertising Agency,Afghan Restaurant,African Restaurant,American Restaurant,Amusement Park,Antique Store,Arcade,Art Gallery,...,Urban Park,Vegan and Vegetarian Restaurant,Video Games Store,Vietnamese Restaurant,Vintage and Thrift Store,Warehouse / Wholesale Store,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant
0,Agincourt,0.000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.04,0.020000,0.00,0.00,0.00,0.00,0.00,0.00
1,Alderwood,0.000,0.00,0.000000,0.00,0.040000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
2,Bathurst Manor,0.000,0.00,0.000000,0.00,0.020000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
3,Beaumond Heights,0.025,0.00,0.000000,0.00,0.025000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
4,Bedford Park,0.000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.02,0.00,0.00,0.00
5,Cabbagetown,0.000,0.00,0.000000,0.00,0.040000,0.00,0.00,0.00,0.00,...,0.02,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
6,Caledonia-Fairbanks,0.000,0.00,0.000000,0.00,0.020000,0.00,0.02,0.00,0.00,...,0.00,0.00,0.00,0.020000,0.00,0.00,0.00,0.00,0.00,0.00
7,Chinatown,0.000,0.02,0.000000,0.00,0.020000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
8,Don Mills North,0.000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00
9,Dorset Park,0.000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.02,0.000000,0.00,0.00,0.00,0.00,0.00,0.02


In [4066]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.24
1  Fast Food Restaurant  0.08
2      Asian Restaurant  0.08
3            Restaurant  0.06
4  Cantonese Restaurant  0.04


----Alderwood----
                           venue  freq
0  Cafes, Coffee, and Tea Houses  0.06
1                         Retail  0.06
2                 Discount Store  0.04
3            American Restaurant  0.04
4                      Drugstore  0.04


----Bathurst Manor----
                         venue  freq
0         Fast Food Restaurant  0.10
1  Grocery Store / Supermarket  0.06
2                         Bank  0.06
3                   Restaurant  0.06
4                    Drugstore  0.04


----Beaumond Heights----
                         venue  freq
0         Fast Food Restaurant  0.18
1         Caribbean Restaurant  0.08
2                    Drugstore  0.08
3  Grocery Store / Supermarket  0.08
4            Indian Restaurant  0.08


----Bedford Park----
                  venue  freq

<h3><b>Correlation: Educational Buildings and Income </b></h3>


In [4006]:
map_clusters_2 = folium.Map(location=[latitude, longitude], zoom_start=10)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for average, lat, lon, neighbourhood, cluster in zip(combined_dataframe["$200,000 and over"], combined_dataframe['Latitude_y'], combined_dataframe['Longitude_y'], combined_dataframe['Neighbourhood'], combined_dataframe['Cluster Labels']):
    label = folium.Popup('Neighbourhood: ' + str(neighbourhood) + " Income: " + str(average), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_2)


map_clusters_2